In [1]:
import torch
torch.set_default_tensor_type(torch.FloatTensor) 
import copy

from rct_data_generator import *
from outcome_models import *
from plotting_functions import *
from mcmc_bayes_update import *
from eig_comp_utils import *
from research_exp_utils import *
from rct_data_generator import generate_host_and_mirror


/Users/lucile/causal_info_gain/pjake/lib/python3.9/site-packages/torch/__init__.py:696: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/tensor/python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


## Set-up

In [2]:
n_host_sample = 100 
sigma_error = 1 
d = 10 
A = torch.randn((d,d))
A = 1/(torch.det(A)) * A

T_allocation_host = torch.randn(d)
T_allocation_host = 100/torch.norm(T_allocation_host)*T_allocation_host

mu_nc = torch.randn(d)
mu_nc = 1/torch.norm(mu_nc)*mu_nc

mu_c = torch.randn(d)
mu_c = 1/torch.norm(mu_c)*mu_c

mu = torch.concat([mu_nc,mu_c])

In [3]:

X_host = (torch.randn((n_host_sample,d)) @ A ) 
T_host = torch.bernoulli(torch.sigmoid(X_host@ T_allocation_host))
X_host_times_T = (T_host.unsqueeze(dim=0).T * X_host)
X_host = torch.concat([X_host,X_host_times_T],dim=1)

Y_host = X_host @ mu
Y_host = (1/Y_host.norm()) * Y_host + sigma_error * torch.randn_like(Y_host)


In [4]:
prior_mean = torch.zeros(2 * d)
beta_0, sigma_0_sq,inv_cov_0 = prior_mean, sigma_error,torch.eye(2*d)
prior_hyperparameters = {'beta_0': beta_0, 'sigma_0_sq': sigma_0_sq,"inv_cov_0":inv_cov_0}
bayesian_regression = BayesianLinearRegression(prior_hyperparameters)
bayesian_regression.set_causal_index(10)

In [5]:
bayesian_regression.fit(X_host,Y_host)

{'posterior_mean': tensor([ 0.2233,  0.2265, -0.1417,  0.0726, -0.3332, -0.1722, -0.2942, -0.1452,
         -0.1194, -0.0381,  0.1880,  0.2240, -0.3335,  0.1367, -0.1961, -0.1363,
         -0.2950, -0.1171,  0.0082, -0.0962]),
 'posterior_cov_matrix': tensor([[ 9.4951e-01, -1.1141e-02, -8.7745e-03, -1.8464e-02,  2.9733e-02,
           1.4875e-02, -1.9158e-02,  1.6286e-02,  2.7513e-02,  7.0635e-03,
          -3.0318e-02, -9.4217e-03, -7.8773e-03, -1.0796e-02,  1.3936e-02,
           3.6941e-03, -6.7244e-03,  8.8176e-03,  1.2982e-02,  3.7619e-03],
         [-1.1141e-02,  9.4890e-01,  9.0535e-03, -1.1704e-02, -4.8561e-03,
           2.6976e-03,  2.0075e-02, -5.6910e-03, -2.1122e-03,  1.3550e-02,
          -9.3713e-03, -3.0418e-02,  1.9708e-03, -1.0242e-02, -2.9833e-03,
          -1.2402e-03,  1.3428e-02, -1.8781e-03, -1.4864e-03,  8.0111e-03],
         [-8.7745e-03,  9.0535e-03,  9.5006e-01,  5.1317e-03, -5.1153e-03,
          -1.4573e-02, -1.8569e-02, -2.6021e-04,  5.8763e-03, -1.1252e-0

In [6]:
bayesian_regression.closed_form_obs_EIG(X_host),bayesian_regression.closed_form_causal_EIG(X_host)

(0.34826900645664116, 0.10570795656747894)

In [7]:
bayesian_regression.samples_obs_EIG(X_host,100,10),bayesian_regression.samples_causal_EIG(X_host,100,10)

(0.19875191273797554, -1.1396286689349324)

## Experiment

In [ ]:
n_host_sample = 100 
sigma_error = 1 
d = 10 
n_repeats = 40

results_dict = {}